In [1]:
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer, TrlParser
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, set_seed
import torch

from peft import PeftModel

from dataclasses import dataclass, field
import argparse

import os
import json
import wandb
import random
import numpy as np

/root/miniconda3/envs/LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/LLM/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/root/miniconda3/envs/LLM/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value T

In [ ]:
## 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,                    ## 4비트 양자화
    bnb_4bit_use_double_quant = True,       ## 추가 양자화로 성능 손실 없이 파라미터당 0.4bit 추가 절약
    bnb_4bit_quant_type = "nf4",            ## 양자화 데이터 타입 지정: 4비트 기반 모델 훈련 시 사용
    bnb_4bit_compute_dtype = torch.bfloat16 ## Llama-3.1-8B의 학습 자료형. 저장은 4비트지만, attention 연산은 해당 포맷으로 역양자화하여 처리
)

## 모델 로드 및 설정
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    device_map = "cuda:0",
    use_cache = False,                          ## VRAM 캐시 미사용, 추론 속도 저하. gradienc_checkpointing과 동시 사용 불가
    low_cpu_mem_usage = True,                   ## CPU RAM 사용량 적게 사용...
    attn_implementation = "flash_attention_2",  ## flash_attention 연산 사용. sdpa가 더 빠르고 효율적일 수도 있음.
    trust_remote_code = True,
    quantization_config = bnb_config,
    dtype = torch.bfloat16                      ## 가중치 로드 데이터 타입. Llama-3.1-8B의 자료형으로 설정
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.01s/it]


KeyError: "attribute 'train' already exists"

In [4]:
## 어뎁터 부착
model = PeftModel.from_pretrained(
    model,
    "results/test/checkpoint-4596",
    is_trainable=True,
    adapter_name="train",
)

model.load_adapter("results/test/checkpoint-4596", adapter_name = "reference")

KeyError: "attribute 'train' already exists"